In [11]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV
import scikitplot as skplt
from sklearn import metrics
from scipy.stats import mannwhitneyu
from sklearn.metrics import confusion_matrix

In [12]:
df1 = pd.read_csv('../ppnnsk.csv')[['Gen', 'die']]
df1 = pd.concat([df1, pd.read_csv('../ppnnw5.csv')[['AATCG', 'ACGAT', 'ATACG', 'ATC', 'ATCA', 'ATCG', 'ATCGA', 'ATTCG', 
                                                  'CAAT', 'CAATC', 'CGAAT', 'CGAT', 'CGATA', 'CGATC', 'CGATT', 'CGTA', 
                                                  'GAT', 'GATC', 'GTCAA', 'TATCG', 'TCAA', 'TCAAT', 'TCGA', 'TCGAT', 'TCGTA', 
                                                  'TGAT']], 
                 pd.read_csv('../ppnnw6.csv')[['ACGATA', 'ATATCG', 'ATCGAT', 'CAATCG', 'CAATTG', 'CCGATA', 'CGATAA', 
                                               'CGATAC', 'CGATAG', 'CGATAT', 'CGATCA', 'CGATTA', 'CGATTG', 'CGCATA', 
                                               'CGGTAT', 'GTCGAT', 'GTCGTA', 'GTTGAT', 'TAATCG', 'TAGTCG', 'TCAATC', 'TCGATA', 
                                               'TCGATC', 'TCGGTA']]], axis = 1)
df1 = df1.loc[df1['die'] == 'no']
df1['Gen'] = 1
df2 = pd.read_csv('../h1sk.csv')[['die']]
df2 = pd.concat([df2, pd.read_csv('../h1w5.csv')[['AATCG', 'ACGAT', 'ATACG', 'ATC', 'ATCA', 'ATCG', 'ATCGA', 'ATTCG', 
                                                  'CAAT', 'CAATC', 'CGAAT', 'CGAT', 'CGATA', 'CGATC', 'CGATT', 'CGTA', 
                                                  'GAT', 'GATC', 'GTCAA', 'TATCG', 'TCAA', 'TCAAT', 'TCGA', 'TCGAT', 'TCGTA', 
                                                  'TGAT']], 
                 pd.read_csv('../h1w6p1.csv')[['ACGATA', 'ATATCG', 'ATCGAT', 'TAATCG', 
                                               'TAGTCG', 'TCAATC', 'TCGATA', 'TCGATC', 'TCGGTA']],
                 pd.read_csv('../h1w6p2.csv')[['CAATCG', 'CAATTG', 'CCGATA', 'CGATAA', 'CGATAC', 'CGATAG', 'CGATAT', 'CGATCA', 
                                               'CGATTA', 'CGATTG', 'CGCATA', 'CGGTAT', 'GTCGAT', 'GTCGTA', 'GTTGAT']]], axis = 1)
df2 = df2.loc[df2['die'] == 'no']
df2['Gen'] = 0

data = pd.concat([df1,df2], axis = 0).reset_index(drop = True)

In [4]:
df3 = pd.read_csv('../Mmu 11k phf2 scored ls')
df3[0] = 0

In [13]:
#Splitting features and labels
X_train = data.iloc[:, 2:].values.astype(float)
y_train = data.iloc[:, 0].values.astype(float)

X_test = df3.iloc[:, :-1].values.astype(float)
y_test = df3.iloc[:, -1].values.astype(float)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 280, criterion = 'entropy')
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

cv_bin = cross_val_score(classifier, X_train, y_train, cv=10, scoring='f1')
cv_mac = cross_val_score(classifier, X_train, y_train, cv=10, scoring='f1_macro')
f1bin = f1_score(y_test, y_pred, average='binary')
f1mac = f1_score(y_test, y_pred, average='macro')


dunnowhat = metrics.accuracy_score(y_test, y_pred)
y_pred_proba = classifier.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

c:\users\michael\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\metrics\_ranking.py:806: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn("No positive samples in y_true, "


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [23]:
cm

array([[11262,    89],
       [    0,     0]], dtype=int64)

In [22]:
f1mac

0.49803210542608234